# Download and convert to Zarr
This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Zarr Database (based on the zcollection package) for future use.
Zarr (and the way we partitionned data with zcollection) is very efficient for computation. However, it is not (yet) compatible with QGIS compared to Geopackage.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
import pixcdust
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon
gdf_geom = gpd.read_file("../data/aoi.gpkg")

dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)

## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=1,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

Downloaded products:   0%|                                                                                    …

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a Zarr (zcollection) database.
This Zarr partionned format is very efficient for time analysis, but is not currently accessible in GIS softwares such as QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.zarr import Nc2ZarrConverter
from glob import glob

In [5]:
pixc = Nc2ZarrConverter(
            glob(pixcdownloader.path_download+'/*/*nc'),
            '/tmp/pixc_zarr',
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

2024-12-13 16:48:21,345 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:32837' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 19), ('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 12), ('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 5), ('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 11), ('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 8), ('concatenate-9ecec8ab536b7b7759ba25b99f1ce62f', 20)} (stimulus_id='handle-worker-cleanup-1734104901.345411')
2024-12-13 16:48:21,351 - distributed.nanny - WARNING - Restarting worker
Exception ignored in: <function CachingFileManager.__del__ at 0x7bc940808280>
Traceback (most recent call last):
  File "/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.

RuntimeError: NetCDF: Can't open HDF5 attribute

database has been succesfully created, we can remove the raw files

In [ ]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
previous steps are not necessary

Now we can open this database in a xarray, or dataframe, or GeoDataFrame

In [ ]:
from pixcdust.readers.zarr import ZarrReader
import datetime

pixc_read = ZarrReader(
    "/tmp/pixc_zarr"
)
pixc_read.read((datetime.datetime(2024,8,3), datetime.datetime(2024,8,12)))
pixc_read.data

In [ ]:
gdf_pixc = pixc_read.to_geodataframe()
gdf_pixc

Enjoy!